# MySQL Database connection

# Import Libraries

In [5]:
#!pip install mysql-connector-python
#!pip install python-dotenv
#!pip install pymysql

In [21]:
import config as cfg
import pandas as pd
import mysql.connector
import os
from datetime import datetime

from dotenv import load_dotenv
load_dotenv("mysql.env")

True

# MySQL connection

In [9]:
HOST=os.environ.get("HOST")
PORT=os.environ.get("PORT")
USER=os.environ.get("USER")
PASSWORD=os.environ.get("PASSWORD")

try: 
    conn = mysql.connector.connect(
        host=HOST,
        port=PORT,
        user=USER,
        password=PASSWORD,
        database="GlobalMarketData"
    )
    query = "SELECT * from histdailyprice3;"
    df = pd.read_sql(query, conn)
    conn.close()
except Exception as e:
    conn.close()
    print(str(e))

# Data

In [10]:
df.head()

,Date,Symbol,Exchange,Close,Open,High,Low,Volume
0,1998-01-01,00001.HK,HKEX,50.750,50.750,50.750,50.750,0
1,1998-01-01,00002.HK,HKEX,34.043,34.043,34.043,34.043,0
2,1998-01-01,00003.HK,HKEX,9.314,9.314,9.314,9.314,0
3,1998-01-01,00004.HK,HKEX,16.821,16.821,16.821,16.821,0
4,1998-01-01,00005.HK,HKEX,63.666,63.666,63.666,63.666,0


In [11]:
print("Dimensions:", df.shape)

Dimensions: (29270724, 8)


In [12]:
df.describe()

,Close,Open,High,Low,Volume
count,2.927072e+07,2.927072e+07,2.927072e+07,2.927072e+07,2.927072e+07
mean,2.706945e+04,2.770904e+04,2.633463e+04,2.692689e+04,2.267096e+06
std,4.412140e+06,4.512667e+06,4.299817e+06,4.393168e+06,2.128552e+07
min,-1.634510e+03,0.000000e+00,-1.764960e+03,0.000000e+00,-2.140735e+09
25%,6.070000e+00,6.190000e+00,5.955000e+00,6.070000e+00,1.450000e+04
50%,1.747000e+01,1.770000e+01,1.721000e+01,1.746000e+01,1.258000e+05
75%,3.479000e+01,3.519000e+01,3.435000e+01,3.478000e+01,7.554000e+05
max,1.555500e+09,1.572000e+09,1.365000e+09,1.536000e+09,1.977816e+10


In [13]:
df.columns

Index(['Date', 'Symbol', 'Exchange', 'Close', 'Open', 'High', 'Low', 'Volume'], dtype='object')

# EDA

## Date

In [14]:
print("Date range:", min(df.Date), "to", max(df.Date))

Date range: 1998-01-01 to 2021-09-07


## Exchange

In [15]:
df.Exchange.unique()

array(['HKEX', 'NYSE', 'NASDAQ', 'AMEX'], dtype=object)

HKEX - The Stock Exchange of Hong Kong Limited <br>
NYSE - The New York Stock Exchange <br>
NASDAQ - Nasdaq Composite <br>
AMEX - American Stock Exchange

## Symbol

In [16]:
pd.Series(df.Symbol.unique())

0        00001.HK
1        00002.HK
2        00003.HK
3        00004.HK
4        00005.HK
           ...   
13908        SLGC
13909       SLGCW
13910       OPP.P
13911       RCO.W
13912        RCOR
Length: 13913, dtype: object

In [17]:
df[df.Symbol == "AAPL"]

,Date,Symbol,Exchange,Close,Open,High,Low,Volume
510,1998-01-01,AAPL,NASDAQ,0.1172,0.1172,0.1172,0.1172,0
2687,1998-01-02,AAPL,NASDAQ,0.1217,0.1451,0.1205,0.1451,25646800
4865,1998-01-05,AAPL,NASDAQ,0.1473,0.1479,0.1356,0.1418,23277200
7043,1998-01-06,AAPL,NASDAQ,0.1423,0.1786,0.1317,0.1691,64766000
9221,1998-01-07,AAPL,NASDAQ,0.1680,0.1697,0.1546,0.1563,37200800
...,...,...,...,...,...,...,...,...
29215717,2021-08-30,AAPL,NASDAQ,149.0000,153.4900,148.6100,153.1200,90956700
29226256,2021-08-31,AAPL,NASDAQ,152.6600,152.8000,151.2900,151.8300,86453100
29236873,2021-09-01,AAPL,NASDAQ,152.8300,154.9800,152.3400,152.5100,80313700
29247492,2021-09-02,AAPL,NASDAQ,153.8700,154.7200,152.4000,153.6500,71171300


# Whether Daily Volatiltiy is above 2%

In [77]:
columns = ["Symbol",
           "Daily Volatility"
          ]
daily_volatility = pd.DataFrame(columns = columns)
daily_volatility

,Symbol,Daily Volatility


In [78]:
symbols = df.Symbol.unique()
print(len(symbols))

13913


In [79]:
for symbol in symbols:
    pct_change = df[df.Symbol == symbol]["Close"].pct_change() * 100
    pct_change.dropna(inplace=True)
    std = pct_change.std()
    daily_volatility = daily_volatility.append({"Symbol": symbol, "Daily Volatility": std}, ignore_index=True)

In [80]:
daily_volatility.head()

,Symbol,Daily Volatility
0,00001.HK,2.208503
1,00002.HK,1.356810
2,00003.HK,2.111346
3,00004.HK,2.500953
4,00005.HK,1.639771


In [81]:
# Join with Exchange
ex = df[["Symbol", "Exchange"]]
ex = ex.groupby("Symbol").first()
daily_volatility = daily_volatility.join(ex, on="Symbol", how="left")
daily_volatility

,Symbol,Daily Volatility,Exchange
0,00001.HK,2.208503,HKEX
1,00002.HK,1.356810,HKEX
2,00003.HK,2.111346,HKEX
3,00004.HK,2.500953,HKEX
4,00005.HK,1.639771,HKEX
...,...,...,...
13908,SLGC,NaN,NASDAQ
13909,SLGCW,NaN,NASDAQ
13910,OPP.P,NaN,NYSE
13911,RCO.W,NaN,AMEX


In [87]:
daily_volatility["Daily Volatility"].isnull().sum()

107

In [89]:
df[df.Symbol == "RCOR"]

,Date,Symbol,Exchange,Close,Open,High,Low,Volume
29265867,2021-09-03,RCOR,AMEX,8.15,8.22,7.49,7.73,117700
29270094,2021-09-07,RCOR,AMEX,7.89,8.59,7.70,8.53,76169


In [82]:
threshold = 2
daily_volatility_above_2 = daily_volatility[daily_volatility["Daily Volatility"] >= threshold]
daily_volatility_above_2

,Symbol,Daily Volatility,Exchange
0,00001.HK,2.208503,HKEX
2,00003.HK,2.111346,HKEX
3,00004.HK,2.500953,HKEX
6,00008.HK,17.548385,HKEX
7,00010.HK,2.377736,HKEX
...,...,...,...
13898,INCR,7.665985,NASDAQ
13899,JXN.P,19.650539,NYSE
13900,LWLG,2.104484,NASDAQ
13901,RICOW,14.269076,NASDAQ


In [90]:
daily_volatility_above_2.to_csv("daily_volatility.csv", index=False)

# Whether Average stock price is below 20%
Check if Average stock price (mode) <br>
If most of the time it is below $20 ["Close"] <br>
Nearest 2 years only

In [43]:
currentYear = datetime.now().year
year_band = currentYear - 2
average = df.copy()
average.Date = pd.to_datetime(average.Date)

,Date,Symbol,Exchange,Close,Open,High,Low,Volume
0,1998-01-01,00001.HK,HKEX,50.7500,50.750,50.750,50.7500,0
1,1998-01-01,00002.HK,HKEX,34.0430,34.043,34.043,34.0430,0
2,1998-01-01,00003.HK,HKEX,9.3140,9.314,9.314,9.3140,0
3,1998-01-01,00004.HK,HKEX,16.8210,16.821,16.821,16.8210,0
4,1998-01-01,00005.HK,HKEX,63.6660,63.666,63.666,63.6660,0
...,...,...,...,...,...,...,...,...
29270719,2021-09-07,ZHDG,AMEX,20.5000,20.560,20.450,20.5400,51255
29270720,2021-09-07,ZIG,AMEX,29.1200,29.120,28.670,28.7200,191481
29270721,2021-09-07,ZOM,AMEX,0.6037,0.610,0.562,0.5805,22233655
29270722,2021-09-07,ZROZ,AMEX,149.5400,149.760,148.650,149.6500,128195


In [13]:
# Close cursor
#cursor.close()

# Close connection
conn.close()

TODO: Check if Average stock price (mode) <br>
If most of the time it is below $20 ["Close"] <br>
Nearest 2 years only

TODO: Replicate GARCH model process on multiple stocks
